# Key Figures Electrical Installations >36kV
Notebook to get key figures of the geodataset Electrical installations with a nominal voltage exceeding 36 kV

In [ ]:
pip install geopandas

In [ ]:
import geopandas as gpd
#import fiona
#import shapely

In [ ]:
url = "https://data.geo.admin.ch/ch.bfe.elektrische-anlagen_ueber_36/gpkg/2056/ElektrischeAnlagenNennspannungUeber36kV_V1.gpkg"
#Leitung = gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Leitung')
#Leitung

In [ ]:
gpd.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Leitung')

Streaming output truncated to the last 5000 lines.
ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NULL in 'OGR_G_GetGeometryType'.

ERROR:fiona._env:Pointer 'hGeom' is NUL

In [ ]:
for layername in fiona.listlayers(url):
    with fiona.open(url, driver="GPKG", layer=layername) as src:
        print(layername, len(src))


ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Leitung 4733
ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Mast 19349
ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Station 323


In [ ]:
df = fiona.read_file(url, driver="GPKG", layer='ElektrischeAnlagenNennspannungUeber36kV_V1_ElektrischeAnlagen_Leitung')

AttributeError: ignored

In [ ]:
df.dtypes

AttributeError: ignored